In [ ]:
# gv fig (u_smth)
from scipy.stats import ttest_1samp

land_feature = cfeature.LAND.with_scale('110m')

varn='gv_v_surf_smth'
# plot for LE

projection = ccrs.Robinson(central_longitude=205)
cbar_kwargs={'orientation':'horizontal','pad':0.05,'shrink':0.7, 'label':' R ', 'aspect':33, 'extend': 'both'}

# 서브플롯 생성
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(18.5, 9), 
             dpi=200, facecolor='w', edgecolor='k', subplot_kw={'projection': projection})

data_array = corr_gv_v_surf_OBS_LE_smth.median(dim='ens_LE')
plt1=data_array.plot.pcolormesh(ax=axes[0, 0], transform=ccrs.PlateCarree(), vmax=1, vmin=-1,cmap=cm.vik, add_colorbar=False)
axes[0, 0].set_title(f'{varn} ((a) LE<->OBS)')
axes[0, 0].add_feature(land_feature, facecolor='gray', edgecolor='none')
    
# plot for (ODA)
data_array = corr_gv_v_surf_OBS_ODA_diff_smth.median(dim='ens_ODA')
plt1=data_array.plot.pcolormesh(ax=axes[0, 1], transform=ccrs.PlateCarree(), vmax=1, vmin=-1,cmap=cm.vik, add_colorbar=False)
axes[0, 1].set_title(f'{varn} ((b) [ODA]<->OBS)')
axes[0, 1].add_feature(land_feature, facecolor='gray', edgecolor='none')

# plot for (ADA)
data_array = corr_gv_v_surf_OBS_ADA_diff_smth.median(dim='ens_ADA')
plt1=data_array.plot.pcolormesh(ax=axes[1, 0], transform=ccrs.PlateCarree(), vmax=1, vmin=-1,cmap=cm.vik, add_colorbar=False)
axes[1, 0].set_title(f'{varn} ((c) [ADA]<->OBS)')
axes[1, 0].add_feature(land_feature, facecolor='gray', edgecolor='none')


# plot for (WND ODA mask)
alpha = 0.10

t_stat_LE, p_values_LE = ttest_1samp(corr_gv_v_surf_OBS_LE_smth, 0, axis=2)
ds_LE = corr_gv_v_surf_OBS_LE_smth.median(dim='ens_LE').where(p_values_LE < alpha)

t_stat_ODA, p_values_ODA = ttest_1samp(corr_gv_v_surf_OBS_ODA_diff_smth, 0, axis=2)
ds_ODA = corr_gv_v_surf_OBS_ODA_diff_smth.median(dim='ens_ODA').where(p_values_ODA < alpha)

t_stat_ADA, p_values_ADA = ttest_1samp(corr_gv_v_surf_OBS_ADA_diff_smth, 0, axis=2)
ds_ADA = corr_gv_v_surf_OBS_ADA_diff_smth.median(dim='ens_ADA').where(p_values_ADA < alpha)

threshold = 0
mask_LE = ds_LE > threshold
mask_ODA = ds_ODA > threshold
mask_ADA = ds_ADA > threshold

ds_all = xr.Dataset(
    data_vars=dict(
        colors=(('lat', 'lon'), np.full(ds_LE.shape, 0.33))  # 'colors'라는 변수 추가
    ),
    coords=dict(
        lat=(('lat'), mask_LE.lat.data),  # DataArray의 .data 사용
        lon=(('lon'), mask_LE.lon.data)     # DataArray의 .data 사용
    )
)

# ds_all['colors'] = xr.where(ds_grid.REGION_MASK == 0, 0.0, ds_all['colors']) # 흰색
ds_all['colors'] = xr.where(mask_LE.isnull(), 0.0, ds_all['colors']) # 흰색
ds_all['colors'] = xr.where(mask_ODA & ~mask_ADA, 0.5, ds_all['colors'])  # 파란색
ds_all['colors'] = xr.where(mask_ODA & mask_ADA, 0.5, ds_all['colors'])  # 파란색
ds_all['colors'] = xr.where(~mask_ODA & mask_ADA, 1.0, ds_all['colors'])  # 주황색
ds_all['colors'] = xr.where(mask_LE & ~mask_ODA, 0, ds_all['colors'])  # 주황색

colors_custom = ['#FFFFFF',  # 흰색 (White)
          '#1E90FF',  # Dodger blue (ODA)
          '#FFD580']  # 파스텔 오렌지 (WND) 

# Define boundaries so each color covers exactly one bin:
bounds = [0, 0.5, 1, 1.5]  # or [0, 0.33, 0.66, 1] if your data is in [0..1]
cmap = mcolors.ListedColormap(colors_custom)
norm = mcolors.BoundaryNorm(bounds, cmap.N)

plt4=ds_all['colors'].plot.pcolormesh(ax=axes[1, 1], transform=ccrs.PlateCarree(), cmap=mcolors.ListedColormap(colors_custom), add_colorbar=False, norm=norm)
axes[1, 1].set_title(f'{varn} (d) Driving Source')
axes[1, 1].add_feature(land_feature, facecolor='gray', edgecolor='none')


# (a), (b), (c) 서브플롯에 대해 위쪽에 큰 컬러바 추가
cbar_a_c = fig.add_axes([0.1, 0.95, 0.55, 0.02])  # [left, bottom, width, height] 비율로 컬러바 위치와 크기 설정
cbar1 = plt.colorbar(plt1, cax=cbar_a_c, orientation='horizontal')
cbar1.set_label('R')

# (d) 서브플롯에 대해 아래쪽에 큰 컬러바 추가
cbar_d = fig.add_axes([0.1, 0.05, 0.55, 0.02])  # [left, bottom, width, height] 비율로 컬러바 위치와 크기 설정
# cbar2= plt.colorbar(plt4, cax=cbar_d, orientation='horizontal')
cbar2 = plt.colorbar(plt4, cax=cbar_d, orientation='horizontal',
                     boundaries=bounds, ticks=[0.25, 0.75, 1.25])
cbar2.set_label('Driving Source')
# num_colors = len(colors_custom)
# tick_locs = np.linspace(0.5 / num_colors, 1 - 0.5 / num_colors, num_colors)
# tick_locs = np.linspace(0.25, 0.75, 3)
# cbar2.set_ticks(tick_locs)  # 눈금 설정
cbar2.set_ticklabels(['RES', 'OCN', 'WND'])  # 눈금 레이블 설정

plt.rcParams.update({'font.size': 16})

# 전체 레이아웃 조정
plt.tight_layout(rect=[0, 0.1, 0.75, 0.9])  # 상단과 하단의 레이아웃 여백을 맞추기 위해 rect 설정
# plt.tight_layout()
plt.savefig(savefilepath +"/BGC_Pred_SFig3_gvu.tif", format='tiff', dpi=150, bbox_inches='tight')
plt.show()

